# Lesson 2: Sequential Chats and Customer Onboarding
## Setup

In [1]:
import os

from autogen import ConversableAgent
from dotenv import load_dotenv

load_dotenv()


llm_config = {
    "model": "gpt-4o-mini",
    "api_key": os.getenv("OPENAI_API_KEY"),
}

## Creating the needed agents

In [14]:
onboarding_personal_information_agent = ConversableAgent(
    name="OnboardingPersonalInformationAgent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="OnboardingTopicPreferenceAgent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="CustomerEngagementAgent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [15]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [16]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
OnboardingPersonalInformationAgent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to OnboardingPersonalInformationAgent):

John

--------------------------------------------------------------------------------
OnboardingPersonalInformationAgent (to customer_proxy_agent):

Thank you, John! And what is your location?

--------------------------------------------------------------------------------
customer_proxy_agent (to OnboardingPersonalInformationAgent):

New York

--------------------------------------------------------------------------------

**************************************************************************

## Print out the summary

In [17]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "name": "John",
  "location": "New York"
}
```


John is interested in reading about soccer.


John is interested in soccer and learns fun facts about the sport, including its ancient origins, the popularity of the FIFA World Cup, the longest recorded match, the origin of the term "soccer," and the fastest goal scored in history. A lighthearted soccer joke is also shared.




In [18]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 5.0849999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.0849999999999996e-05, 'prompt_tokens': 203, 'completion_tokens': 34, 'total_tokens': 237}}, 'usage_excluding_cached_inference': {'total_cost': 5.0849999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.0849999999999996e-05, 'prompt_tokens': 203, 'completion_tokens': 34, 'total_tokens': 237}}}


{'usage_including_cached_inference': {'total_cost': 1.7699999999999997e-05, 'gpt-4o-mini-2024-07-18': {'cost': 1.7699999999999997e-05, 'prompt_tokens': 82, 'completion_tokens': 9, 'total_tokens': 91}}, 'usage_excluding_cached_inference': {'total_cost': 1.7699999999999997e-05, 'gpt-4o-mini-2024-07-18': {'cost': 1.7699999999999997e-05, 'prompt_tokens': 82, 'completion_tokens': 9, 'total_tokens': 91}}}


{'usage_including_cached_inference': {'total_cost': 0.0003246, 'gpt-4o-mini-2024-07-18': {'cost': 0.0003246, 'prompt_tokens': 544, 'completion_tokens': 405, 'total_tokens': 949}}, 'usa